In [156]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility')
from notebooks.Will.toolbox.IOs.get_calculated_transforms import get_demons_transform
import SimpleITK as sitk


In [2]:
demons_transform = get_demons_transform('DK39')

In [4]:
print(demons_transform)

itk::simple::Transform
 GaussianSmoothingOnUpdateDisplacementFieldTransform (0x2c72390)
   RTTI typeinfo:   itk::GaussianSmoothingOnUpdateDisplacementFieldTransform<double, 3u>
   Reference Count: 1
   Modified Time: 682
   Debug: Off
   Object Name: 
   Observers: 
     none
   DisplacementField: 
     Image (0x361a290)
       RTTI typeinfo:   itk::Image<itk::Vector<double, 3u>, 3u>
       Reference Count: 3
       Modified Time: 679
       Debug: Off
       Object Name: 
       Observers: 
         none
       Source: (none)
       Source output name: (none)
       Release Data: Off
       Data Released: False
       Global Release Data: Off
       PipelineMTime: 0
       UpdateMTime: 0
       RealTimeStamp: 0 seconds 
       LargestPossibleRegion: 
         Dimension: 3
         Index: [0, 0, 0]
         Size: [435, 262, 117]
       BufferedRegion: 
         Dimension: 3
         Index: [0, 0, 0]
         Size: [435, 262, 117]
       RequestedRegion: 
         Dimension: 3
         

In [53]:
import numpy as np
import plotly.graph_objects as go

In [6]:
Size = np.array([435, 262, 117])
Spacing = np.array([41.6, 41.6, 80])
Origin = np.array([31.2, 26, 40])


In [78]:
demons_transform.TransformPoint([100,100,100])

(101.38200871177668, 100.35988824286947, 100.15685331431678)

In [142]:
def get_point_list(grid):
    x,y,z = grid.shape[1:]
    point_list = []
    for xi in range(x):
        for yi in range(y):
            for zi in range(z):
                point_list.append(grid[xi,yi,zi])
    return point_list 

def transform_grid(grid):
    transformed_grid = np.zeros(grid.shape)
    x,y,z = grid.shape[1:]
    for xi in range(x):
        for yi in range(y):
            for zi in range(z):
                pointi = grid[:,xi,yi,zi]
                transformed_pointi = demons_transform.TransformPoint(pointi)
                transformed_grid[:,xi,yi,zi] = transformed_pointi
    return transformed_grid

def get_changed_points(grid_diff):
    # changed_grid = np.zeros(grid_diff.shape)
    changed_points = []
    x,y,z = grid.shape[1:]
    for xi in range(x):
        for yi in range(y):
            for zi in range(z):
                pointi = grid[:,xi,yi,zi]
                if np.any(pointi!=0):
                    changed_points.append([xi,yi,zi])
    return np.asarray(changed_points)


In [143]:
xstart = -500
ystart = -500
zstart = -500
xend = 1000
yend = 1000
zend = 1000
ndots = 20
grid = np.mgrid[xstart:xend:(xend-xstart)/ndots,ystart:yend:(yend-ystart)/ndots,zstart:zend:(zend-zstart)/ndots]

In [144]:
transformed_grid = transform_grid(grid)

In [145]:
grid_diff= transformed_grid - grid
changed_points = get_changed_points(grid_diff)


In [147]:
changed_points.shape

(8000, 3)

In [154]:
np.max(changed_points,axis=0)

array([19, 19, 19])

In [157]:
sitk.DisplacementFieldJacobianDeterminant(demons_transform)

TypeError: in method 'DisplacementFieldJacobianDeterminant', argument 1 of type 'itk::simple::Image const &'